In [1]:
!pip install accelerate==0.21.0 


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip install peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 

  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.7.0 requires transformers<5.0.0,>=4.34.0, but you have transformers 4.31.0 which is incompatible.

[notice] A new

In [3]:
!pip install -r requirements.txt

  Using cached bitsandbytes-0.41.2-py3-none-any.whl.metadata (9.8 kB)
  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached peft-0.6.0-py3-none-any.whl.metadata (23 kB)
  Using cached accelerate-0.25.0-py3-none-any.whl.metadata (18 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached tokenizers-0.14.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached trl-0.8.3-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
  Using cached packaging-23.1-py3-none-any.whl.metadata (3.1 kB)
Using cached bitsandbytes-0.41.2-py3-none-any.whl (92.6 MB)
Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
Using cached peft-0.6.0-py3-none-any.whl (134 kB)
Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
Using cached tokenizers-0.14.1-cp311-cp311-manylinux_2_17_x86_64.

In [4]:
!pip install peft



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, TaskType, get_peft_model
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

2024-05-12 21:53:48.403625: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 21:53:48.425145: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 21:53:48.425164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 21:53:48.425937: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 21:53:48.430573: I tensorflow/core/platform/cpu_feature_guar

In [6]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
import pandas as pd
file_path = "QUestion_Answer_pairs.csv"
combined_df = pd.read_csv(file_path)


# Define a list to store transformed data
transformed_data = []
combined_df

# Define a function to transform each row
def transform_conversation(row):
    question_text = row['question']
    answers_text = row['answer']

    # Apply the new template and append to the list
    transformed_data.append({'text': f'<s>[INST] {question_text} [/INST] {answers_text} </s>'})

# Apply the transformation to each row
_ = combined_df.apply(transform_conversation, axis=1)

# Convert the list of dictionaries to a DataFrame
transformed_df = pd.DataFrame(transformed_data)

from datasets import Dataset
# Create a Dataset object with the DataFrame
dataset = Dataset.from_pandas(transformed_df)


In [8]:
from sklearn.model_selection import train_test_split

# Assuming transformed_dataset is a DataFrame
transformed_train_df, transformed_test_val_df = train_test_split(transformed_df, test_size=0.3, random_state=42)
transformed_val_df, transformed_test_df = train_test_split(transformed_test_val_df, test_size=0.5, random_state=42)

from datasets import Dataset
# Create a Dataset object with the DataFrame

transformed_train_dataset = Dataset.from_pandas(transformed_train_df)
transformed_val_dataset = Dataset.from_pandas(transformed_val_df)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
# dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    evaluation_strategy="steps",  # Evaluate every 'logging_steps', could also be "epoch"
    eval_steps=logging_steps,  
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_train_dataset,
    eval_dataset=transformed_val_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/5225 [00:00<?, ? examples/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

wandb: Currently logged in as: pavanvicky2015 (sjsuedu). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
25,4.484200,4.347180


In [ ]:
import wandb
wandb.finish()
trainer.model.save_pretrained("Llama2_7B_PEFT_final_2")

In [ ]:
from peft import LoraConfig, PeftModel
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, "Llama2_7B_PEFT_final")
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
pavanvicky1998/Group5_NewsChatBot

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("google-bert/bert-base-cased")

# Push the model to your namespace with the name "my-finetuned-bert".
model.push_to_hub("my-finetuned-bert")

# Push the model to an organization with the name "my-finetuned-bert".
model.push_to_hub("huggingface/my-finetuned-bert")

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import create_repo
create_repo("group5_newsdata", private = False)

In [ ]:
model.push_to_hub("group5", check_pr=True)

tokenizer.push_to_hub("group5",check_pr=True)

In [ ]:
from huggingface_hub import Repository

# Create a repository object for the destination repository
repo = Repository("group5_newsdata")

# Push your model to the repository
# repo.push("Llama2_7B_PEFT_final")


In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "Llama2_7B_PEFT_final"
repo_name = get_full_repo_name(model_name)
repo_name

In [ ]:
output_dir = "Llama2_7B_PEFT_final"
repo = Repository(output_dir, clone_from=repo_name)